In [ ]:
# Librairies de base
import numpy as np
import pandas as pd

# Librairies pour ML
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# Optionnel pour l'arbre de décision classique
from sklearn.tree import DecisionTreeClassifier


In [ ]:
#1-chargement des données
data =pd.read_csv('data.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
data = data.drop(['Unnamed: 32'], axis=1)
print(data.columns)

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [ ]:
X = data.drop(['diagnosis'],axis=1)
y = data['diagnosis']
# Si y contient 'B' et 'M'
y = np.array([0 if val == 'B' else 1 for val in y])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#6.division des données trainset(70%) et testset(30%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 0)

In [ ]:
class TreeEnv:
    def __init__(self, X, y):
        # Convertir X et y en NumPy arrays de type float/int
        self.X = np.array(X, dtype=float)
        self.y = np.array(y, dtype=int)
        self.n_features = self.X.shape[1]

    def get_possible_actions(self, indices):
        actions = []
        for f in range(self.n_features):
            thresholds = np.unique(self.X[indices, f])
            for t in thresholds:
                actions.append((f, t))
        return actions

    def step(self, indices, action):
        indices = np.array(indices)
        f, t = action
        left_indices = indices[self.X[indices, f] <= t]
        right_indices = indices[self.X[indices, f] > t]
        reward = self.information_gain(indices, left_indices, right_indices)
        return left_indices, right_indices, reward

    def information_gain(self, parent_indices, left_indices, right_indices):
        def entropy(indices):
            if len(indices) == 0:
                return 0
            p = np.mean(self.y[indices])
            if p == 0 or p == 1:
                return 0
            return -p*np.log2(p) - (1-p)*np.log2(1-p)

        n = len(parent_indices)
        n_left = len(left_indices)
        n_right = len(right_indices)

        gain = entropy(parent_indices) - (n_left/n)*entropy(left_indices) - (n_right/n)*entropy(right_indices)
        return gain


In [ ]:
# Paramètres
alpha = 0.1      # taux d'apprentissage
gamma = 0.9      # facteur de discount
epsilon = 0.1    # exploration
n_episodes = 200

# Convertir train set en NumPy arrays si nécessaire
X_train_np = X_train if isinstance(X_train, np.ndarray) else X_train.values
y_train_np = y_train if isinstance(y_train, np.ndarray) else y_train.values

env = TreeEnv(X_train_np, y_train_np)

# Q-table : dictionnaire (state, action) -> valeur
Q = {}

# Q-learning
for episode in range(n_episodes):
    indices = np.arange(len(y_train_np))
    done = False

    while not done:
        state = tuple(indices)
        actions = env.get_possible_actions(indices)
        if not actions:
            break

        # epsilon-greedy
        if np.random.rand() < epsilon:
            action = actions[np.random.randint(len(actions))]
        else:
            q_vals = [Q.get((state, a), 0) for a in actions]
            action = actions[np.argmax(q_vals)]

        left, right, reward = env.step(indices, action)

        # Next state max Q
        max_next = 0
        for next_indices in [left, right]:
            for a_next in env.get_possible_actions(next_indices):
                max_next = max(max_next, Q.get((tuple(next_indices), a_next), 0))

        # Update Q
        Q[(state, action)] = Q.get((state, action), 0) + alpha * (reward + gamma*max_next - Q.get((state, action), 0))

        # Continuer avec le noeud le plus grand
        indices = left if len(left) >= len(right) else right

        if len(left) == 0 or len(right) == 0:
            done = True

# -------------------------------


In [ ]:
# Construction de l'arbre à partir de Q-table
# -------------------------------

class QDecisionTreeNode:
    def __init__(self, indices, depth=0, max_depth=5):
        self.indices = indices
        self.depth = depth
        self.max_depth = max_depth
        self.left = None
        self.right = None
        self.feature = None
        self.threshold = None
        self.label = None

    def build(self, env, Q):
        y_node = env.y[self.indices]
        if len(np.unique(y_node)) == 1 or self.depth >= self.max_depth:
            self.label = np.round(np.mean(y_node))
            return

        state = tuple(self.indices)
        actions = env.get_possible_actions(self.indices)
        if not actions:
            self.label = np.round(np.mean(y_node))
            return

        # Action avec Q max
        q_vals = [Q.get((state, a), 0) for a in actions]
        best_action = actions[np.argmax(q_vals)]
        self.feature, self.threshold = best_action

        left_indices = self.indices[env.X[self.indices, self.feature] <= self.threshold]
        right_indices = self.indices[env.X[self.indices, self.feature] > self.threshold]

        if len(left_indices) == 0 or len(right_indices) == 0:
            self.label = np.round(np.mean(y_node))
            return

        self.left = QDecisionTreeNode(left_indices, self.depth+1, self.max_depth)
        self.left.build(env, Q)
        self.right = QDecisionTreeNode(right_indices, self.depth+1, self.max_depth)
        self.right.build(env, Q)



In [ ]:
# Fonction de prédiction
def predict_tree(node, X):
    preds = []
    for x in X:
        current = node
        while current.label is None:
            if x[current.feature] <= current.threshold:
                current = current.left
            else:
                current = current.right
        preds.append(current.label)
    return np.array(preds)



In [ ]:
# Construire arbre
root = QDecisionTreeNode(indices=np.arange(len(y_train_np)), depth=0, max_depth=5)
root.build(env, Q)

# Prédiction
X_test_np = X_test if isinstance(X_test, np.ndarray) else X_test.values
y_pred = predict_tree(root, X_test_np)

# Évaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8421052631578947
Confusion Matrix:
 [[93 15]
 [12 51]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.86      0.87       108
           1       0.77      0.81      0.79        63

    accuracy                           0.84       171
   macro avg       0.83      0.84      0.83       171
weighted avg       0.84      0.84      0.84       171

